# Implementation of LSTM model with Convolution Layers

- Architecture
    - Conv1d -> LSTM (l1) -> LSTM (l2) -> FC -> FC -> FC

- Initial Params
    - convlution layer, filters = 64, kernel_size = 3, strides = 1, activation = "relu", padding = "causal", input_shape = [window_size, 1]
    - lstm layer 1; units = 64, return_sequences = True 
    - lstm layer 2; units = 64
    - Dense layers; units = 30, activation = "relu"
    - Dense layer; units = 30, activation = "relu"

In [6]:
import torch 
import torch.nn as nn 
import torch.optim as optim 

import numpy as np 
import pandas as pd 

import sys 
import os 

import plotly.graph_objs as go 

sys.path.append("..")
from utils.utils import windowed_dataset, generate_cyclic_features, ohe, train_val_test_split, normalise, torch_dataset, inverse_normalise

## Read Data

In [23]:
# Read data 
df = pd.read_csv("../data/load_ammended.csv")
df.head()

,day_of_week,hour,energy_load
0,0,19,447
1,0,20,435
2,0,21,451
3,0,22,442
4,0,23,444


## Params

In [33]:
window_size = 24
batch_size = 64

val_ratio = 0.15
test_ratio = 0.15

## Preprocessing

In [25]:
# Generate sin_hour, cos_hour
# Replace hour with these components as NN will inherently learn better.
df = generate_cyclic_features(df, "hour", 24)

# One hot encode day_of_week
ohe_arr = ohe(df, ["day_of_week"])

# Windowed dataset - removes last incomplete window
X,y = windowed_dataset(seq = df["energy_load"], ws= window_size)

# Remove the last incompleted window (Since the windowed dataset removes incompleted window)
ohe_arr = ohe_arr[:len(X)]

### Feature Concatenation
**Approach 1**
- X = [[x1, ..., x24, d1,0,..., 0, s1,c1];[x2, ..., x25,0,d2, ...,0,s2,c2]; ...]
- shape : (df.shape[0], 33) -> later once batched : (batch_size, 33)

In [26]:
# Stack features
X = np.hstack((X, ohe_arr))
print(f"X : {X.shape}")

X : (3451, 33)


### Train-Val-Test Split

In [29]:
# Train - Validation - Test Split 
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y, val_ratio, test_ratio)

print("shapes") 
print(f"X_train : {X_train.shape}") 
print(f"y_train : {y_train.shape}") 
print(f"X_val : {X_val.shape}") 
print(f"y_val : {y_val.shape}") 
print(f"X_test : {X_test.shape}") 
print(f"y_test : {y_test.shape}") 

shapes
X_train : (2493, 33)
y_train : (2493,)
X_val : (440, 33)
y_val : (440,)
X_test : (518, 33)
y_test : (518,)


### Normalise

In [31]:

# Normalise Data
(norm_data, normaliser) = normalise(X_train, X_val, X_test, y_train.reshape(-1,1), y_val.reshape(-1,1), y_test.reshape(-1,1))

X_train_norm, X_val_norm, X_test_norm, y_train_norm, y_val_norm, y_test_norm = norm_data

### Torch Datasets

In [34]:
# Get torch datasets
train_loader, val_loader, test_loader = torch_dataset(
    X_train_norm, 
    X_val_norm, 
    X_test_norm, 
    y_train_norm, 
    y_val_norm, 
    y_test_norm,
    batch_size = batch_size
)

## Model

In [36]:
class LSTMHybrid(nn.Module):
    def __init__(self):
        super(LSTMHybrid,self).__init__()
        pass

    def forward(self,X):
        pass